# Setup

In [ ]:
!pip install transformers
!pip install deepspeed

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 28.5 MB/s 
     |████████████████████████████████| 3.3 MB 12.9 MB/s 
     |████████████████████████████████| 636 kB 37.0 MB/s 
     |████████████████████████████████| 52 kB 999 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 477 kB 5.5 MB/s 
     |████████████████████████████████| 216 kB 45.6 MB/s 
     |████████████████████████████████| 108 kB 52.1 MB/s 
     |████████████████████████████████| 15.2 MB 38 kB/s 
  Created wheel for deepspeed: filename=deepspeed-0.5.3-py3-none-any.whl size=480187 sha256=be38f51c1d9f0cfd99d637e45ea908cd00f81533ae717f19331072e821c6710b
  Stored in directory: /root/.cache/pip/wheels/8f/a3/c9/2d8777fa5b18c86e8d1dd2d8197c13596ae14ac5996fb329be
Successfully built deepspeed


## Mounting drive files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '9994'
os.environ['RANK'] = "0"
os.environ['LOCAL_RANK'] = "0"
os.environ['WORLD_SIZE'] = "1"


## Importing libraries

In [ ]:
import pandas as pd
import torch
from numba import cuda
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPTNeoForCausalLM
import re

In [ ]:
torch.manual_seed(42)
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B", bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>', pad_token='<|pad|>')

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/200 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").cuda()
model.resize_token_embeddings(len(tokenizer))


Downloading:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

In [ ]:
descriptions = pd.read_csv('/content/drive/MyDrive/proyectos/ia/recipie/full_dataset.csv')

In [ ]:
df = descriptions[['ingredients','directions','title']]

names = list(df.columns)

max_length=1000

In [ ]:
df.head()

,ingredients,directions,title
0,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",No-Bake Nut Cookies
1,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",Jewell Ball'S Chicken
2,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",Creamy Corn
3,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",Chicken Funny
4,"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",Reeses Cups(Candy)


# Testing model

In [ ]:
class Recetas(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list.iterrows():
          texto = ""
          for i,part in enumerate(txt[1]):            
            texto+=names[i]+'\n'+part+'\n'
          texto=re.sub('\"\s?,\s?\"','\n',texto)
          texto=re.sub('\"|\[|\]','',texto)
          #print(texto)
          

          encodings_dict = tokenizer('<|startoftext|>' + texto + '<|endoftext|>', truncation=True,
                                    max_length=max_length, padding="max_length")
          self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
          self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
dataset = Recetas(df, tokenizer, max_length=max_length)
torch.cuda.empty_cache()

In [ ]:
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
training_args = TrainingArguments(output_dir='./results', num_train_epochs=5, logging_steps=5000, save_steps=5000,
                                  per_device_train_batch_size=1, per_device_eval_batch_size=1,
                                  warmup_steps=100, weight_decay=0.01, logging_dir='./logs')

In [ ]:
Trainer(model=model, args=training_args, train_dataset=train_dataset,
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

RuntimeError: ignored

In [ ]:
torch.cuda.empty_cache()

In [ ]:
predict= """<|startoftext|>
ingredients
1p apple 
1p lemon
"""
generated = tokenizer(predict, return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(generated, do_sample=True, top_k=50,
                                max_length=1000, top_p=0.95, temperature=1.9, num_return_sequences=20)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))